### 파이썬이 패키지를 찾을 때, sys.path 탐색
* 그래서 ViT 경로 넣어줌 models 패키지(라이브러리) 찾을 수 있게
* <a href="https://devvvyang.tistory.com/31">관련 포스팅</a>

In [42]:
import sys
# sys.path.append('G:\\Doby\\ViT')
print(sys.path)

['G:\\Doby\\ViT\\models\\transformer_encoder', 'C:\\Users\\user\\anaconda3\\python310.zip', 'C:\\Users\\user\\anaconda3\\DLLs', 'C:\\Users\\user\\anaconda3\\lib', 'C:\\Users\\user\\anaconda3', '', 'C:\\Users\\user\\anaconda3\\lib\\site-packages', 'C:\\Users\\user\\anaconda3\\lib\\site-packages\\win32', 'C:\\Users\\user\\anaconda3\\lib\\site-packages\\win32\\lib', 'C:\\Users\\user\\anaconda3\\lib\\site-packages\\Pythonwin', 'G:\\Doby\\ViT']


In [52]:
import torch
from torch import nn
from torchinfo import summary
from models.attention.multi_head_attention import MultiHeadAttention

In [64]:
class TransformerEncoder(nn.Module):
    def __init__(self, embedding_size, d_dim, qkv_dim, head_num, hidden_dim):
        '''
        embedding_size : 임베딩의 개수
        d_dim : 현재 임베딩의 차원 수
        qkv_dim : attention 후에 임베딩의 차원 수 (현재 같아야 하는 것으로 추측)
        head_num : Multi-Head Self-Attention에서 사용되는 Attention의 개수
        hidden_dim : MLP에 사용되는 Hidden Layer의 Dimension
        n_transformers : transformer layer의 수
        
        Input Shape : (embedding_size, d_dim)
        Multi-Head Self-Attention Shape : (embedding_size, qkv_dim)
        qkv_dim과 d_dim이 같아야 residual connection이 가능하다.
        나의 오류일 수도 있으니 d_dim, qkv_dim 인자는 우선 구분해두자.

        -> Input Shape, Output Shape가 같다.
        '''
        
        super().__init__()
        self.layer_norm1 = nn.LayerNorm(d_dim)
        self.mhatt = MultiHeadAttention(d_dim, qkv_dim, head_num)

        self.layer_norm2 = nn.LayerNorm(qkv_dim)
        self.MLP = nn.Sequential(nn.Linear(qkv_dim, hidden_dim),
                                 nn.GELU(),
                                 nn.Linear(hidden_dim, qkv_dim))

    def forward(self, x):
        residual = x
        x = self.layer_norm1(x)
        x = self.mhatt(x)
        x = x + residual

        residual = x
        x = self.layer_norm2(x)
        x = self.MLP(x)
        x = x + residual

        return x

In [65]:
trs = TransformerEncoder(30, 10, 10, 8, 20)

In [66]:
summary(trs, (1, 30, 10))

Layer (type:depth-idx)                   Output Shape              Param #
TransformerEncoder                       [1, 30, 10]               --
├─LayerNorm: 1-1                         [1, 30, 10]               20
├─MultiHeadAttention: 1-2                [1, 30, 10]               800
├─LayerNorm: 1-3                         [1, 30, 10]               20
├─Sequential: 1-4                        [1, 30, 10]               --
│    └─Linear: 2-1                       [1, 30, 20]               220
│    └─GELU: 2-2                         [1, 30, 20]               --
│    └─Linear: 2-3                       [1, 30, 10]               210
Total params: 1,270
Trainable params: 1,270
Non-trainable params: 0
Total mult-adds (M): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 0.01
Estimated Total Size (MB): 0.02